In [1]:
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


You should consider upgrading via the 'D:\projects\vsCode\pythonForHtests\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip3 install -q "monai-weekly[nibabel, skimage, pillow, tensorboard, gdown, ignite, torchvision, itk, tqdm, lmdb, psutil,  openslide, pandas, einops, transformers, mlflow, matplotlib, tensorboardX, tifffile, imagecodecs]"

You should consider upgrading via the 'D:\projects\vsCode\pythonForHtests\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip3 install h5py

You should consider upgrading via the 'D:\projects\vsCode\pythonForHtests\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
from monai.config import print_config
print_config()

MONAI version: 0.9.dev2210
Numpy version: 1.21.5
Pytorch version: 1.10.2+cu113
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 1a660e6a7a50e985af5ff76b559baab44175438c
MONAI __file__: d:\projects\vsCode\pythonForHtests\venv\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.8
Nibabel version: 3.2.2
scikit-image version: 0.19.2
Pillow version: 9.0.1
Tensorboard version: 2.8.0
gdown version: 4.4.0
TorchVision version: 0.11.3+cu113
tqdm version: 4.63.0
lmdb version: 1.3.0
psutil version: 5.9.0
pandas version: 1.1.5
einops version: 0.4.1
transformers version: 4.17.0
mlflow version: 1.24.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [5]:
import numpy as np

from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,

)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob



from monai.transforms import (
    LoadImage, LoadImaged, EnsureChannelFirstd,
    Resized,  Compose
)
from monai.config import print_config
import re


In [6]:
from monai.transforms import *

set_determinism(seed=0)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Spacingd(keys=["image", "label"], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        EnsureTyped(keys=["image", "label"]),
    ]
)


In [7]:
data_dir = "D:\\dataSets\\CTORGmini\\"

train_images = sorted(
    glob.glob(os.path.join(data_dir, "volumes 0-49", "*.nii.gz")))

train_labels = sorted(
    glob.glob(os.path.join(data_dir, "labels", "*.nii.gz")))

data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(train_images, train_labels)
]
data_dicts

[{'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-20.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-20.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-21.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-21.nii.gz'}]

In [8]:
def getNumberFromFilePath(dat):
    string = str(dat['image_meta_dict']['filename_or_obj'])
    numb = re.search('.nii', string).span()[0] ## this does what you wanted
    describingNumber = string[(numb-2):(numb)].replace("-","")
    return int(describingNumber)

In [9]:
import h5py
f = h5py.File("D:\\dataSets\\forMainHDF5\\forJuliaExample.hdf5", "w")

In [10]:
#we are looking for a mask with given organ and save a boolean that will make later benchmarks easier
def processGivenMaskAndSaveSum(grp, name, dat,numb ):
    labelBoolTensor =torch.where( dat['label']==numb, 1, 0).bool()
    summ= torch.sum(labelBoolTensor)
    #if summ>0:
    labelBoolDataset = grp.create_dataset(name, data=  torch.flatten(labelBoolTensor).detach().numpy())
    labelBoolDataset.attrs['sum']=  summ
    #return summ

In [ ]:
check_ds = Dataset(data=data_dicts, transform=val_transforms)
check_loader = DataLoader(check_ds, batch_size=1)

for dat in check_loader:
    describingNumber = getNumberFromFilePath(dat)
    #group collecting
    grp = f.create_group(str(describingNumber))
    
    arrSums= np.zeros(7)
    for maskNumb in range(1,7) :
        locSum= processGivenMaskAndSaveSum(grp, str(maskNumb), dat,maskNumb )
        arrSums[maskNumb]=maskNumb
    for maskNumb in range(1,7) :
        getSomeHausdorffDist(grp, str(describingNumber), dat,maskNumb , arrSums  )
    print(describingNumber)
    pass

KeyboardInterrupt: 

In [12]:
f.close()